In [17]:
import asyncio
from pyppeteer import launch
import nest_asyncio
nest_asyncio.apply()
import asyncio
import json  

In [18]:
import urllib
import requests
from bs4 import BeautifulSoup

In [21]:
import random

### 获取cookies

In [3]:
headers ={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cache-Control': 'max-age=0',
'Referer': 'https://www.nytimes.com/',
'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
'Sec-Ch-Ua-Mobile': '?0',
'Sec-Ch-Ua-Platform': '"Windows"',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

In [12]:
async def get_cookies():
    browser = await launch(headless=False,dumpio=True,
                               executablePath=r'C:\Program Files\Google\Chrome\Application\chrome.exe',
                               # 可以指定本地chrome浏览器的用户文件所在地，也可以建一个空文件夹,目的是为下一步采用本地chrom登录页面做准备。
                               userDataDir=r"C:\Users\Administrator\Desktop\userdatadir", 
                               args=['--no-sandbox', '--disable-infobars']) 
    
    page = await browser.newPage()
    await page.setExtraHTTPHeaders(headers)

    # 导航到登录页面，默认登录或者手动登录，默认登录可以采用本地保存了密码的chrom。如果采用代码模拟登录，容易被封ip
    # 网易云登录，直接扫码登录即可，留的时间应该足够扫码了
    await page.goto('https://music.163.com/')
    await page.evaluate( '''() =>{ Object.defineProperties(navigator,{ webdriver:{ get: () => false } }) }''')
    await page.setViewport({'width': 1280, 'height': 800})
    await page.waitFor(3000)
    await asyncio.sleep(10)
    
    # 打开官网页面，获取cookies
    page2 = await browser.newPage()
    await page2.setViewport(viewport={'width': 1280, 'height':800})
    await page2.goto('https://music.163.com/#/user/home?id=482487093')
    await page2.waitFor(3000) 
    await asyncio.sleep(10)
    
    
    # 获取页面的cookies并解析为request能够使用的形式
    cookies=await page2.cookies()
    print(cookies)
    
    fil_cookies = { cookie['name']: cookie['value'] for cookie in cookies}
    with open(r'fil_cookies_qq.json',"w") as file:
        json.dump(fil_cookies, file)
        
    await page.waitFor(3000)
    await browser.close()
    
if __name__ == '__main__':
    
    loop = asyncio.get_event_loop()
    loop.run_until_complete(get_cookies())

[{'name': 'ntes_kaola_ad', 'value': '1', 'domain': '.music.163.com', 'path': '/', 'expires': 1745247730.883943, 'size': 14, 'httpOnly': False, 'secure': False, 'session': False, 'priority': 'Medium', 'sameParty': False, 'sourceScheme': 'Secure', 'sourcePort': 443}, {'name': 'MUSIC_U', 'value': '0058C5BD19941C6BD285F1A002EF60D3E0C0A543A495F4346B5E5816A40723EBF468A3BC2200B93B490E402D820EA7A57F59054935AF2A26411E9AC8FA1AD907D2818C0CDC04014A73D3FF3E16CF74D0963F943D84F5F3B24E99AEF58FCDD8D36FA6B42066A5C3602CD55E282B86780188C4A5E76134CBEDBF697E76A11994F586534D68BA13A181D24A8D10F2848BA97509F2CC9EBB13C3E57AEE4B612A9734345072513B0C12A7F148F8E21475190A9BE6CEE166EFEF3317519D232A2C3CDAF7E3FFD7D9283196A41415E503401C012662EA02609CE9A18B4B19502450B5CF7ED657A8C18E600C660CB251EEA93F2D38840305DC9C9ED5B6A2B5ACCEA828E5FF26B4DFD23E607BF0DB471CC06A7AED4C4786E43B615E09B188BD03AAA5EC46247C62443EE50BCE827BF294BFB5858D43EE6DA84F0BFDE85780DBA73A46CBE978DAD9FA851427D131E507DB2939CCA82A88573EEF441C325FA9BE019D06D7D6

### 测试某网易云音乐页面是否为用户

In [14]:
# 将cookies用json保存到本地后调用

with open('fil_cookies_qq.json', 'r', encoding = 'utf-8') as f:
    cookies = json.load(f)
cookies

{'ntes_kaola_ad': '1',
 'MUSIC_U': '0058C5BD19941C6BD285F1A002EF60D3E0C0A543A495F4346B5E5816A40723EBF468A3BC2200B93B490E402D820EA7A57F59054935AF2A26411E9AC8FA1AD907D2818C0CDC04014A73D3FF3E16CF74D0963F943D84F5F3B24E99AEF58FCDD8D36FA6B42066A5C3602CD55E282B86780188C4A5E76134CBEDBF697E76A11994F586534D68BA13A181D24A8D10F2848BA97509F2CC9EBB13C3E57AEE4B612A9734345072513B0C12A7F148F8E21475190A9BE6CEE166EFEF3317519D232A2C3CDAF7E3FFD7D9283196A41415E503401C012662EA02609CE9A18B4B19502450B5CF7ED657A8C18E600C660CB251EEA93F2D38840305DC9C9ED5B6A2B5ACCEA828E5FF26B4DFD23E607BF0DB471CC06A7AED4C4786E43B615E09B188BD03AAA5EC46247C62443EE50BCE827BF294BFB5858D43EE6DA84F0BFDE85780DBA73A46CBE978DAD9FA851427D131E507DB2939CCA82A88573EEF441C325FA9BE019D06D7D603',
 'gdxidpyhxdE': 'uxVCiaIgSYgJNV%2FY3%5CeI7hp7YheNp2fVctlI5jRaE%5Ck2nW9lo%5CVf2DT3ZxMKVq4Vp%2B1rPhruXptQZEJBbd2fC0Ns4kinV9iVT0UC0W1zk7m1pv%2BE6czON8aibtCGiMM1bVjD5kuL%2FNNnVp%2F3V8EzL8aDo0C00X3BCqoisKG%2Bj8oDVlvs%3A1710688125637',
 'sDeviceId': 'YD-JZWLdK0

In [15]:
headers= {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    "pragma": "no-cache",
        # 加'\'转义，确保值正确。
    "sec-ch-ua": "\"Google Chrome\";v=\"117\", \"Not;A=Brand\";v=\"8\", \"Chromium\";v=\"117\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
   }

In [19]:
# 一定要用'http://music.163.com/user/home?id=482487093'这个形式
url = 'http://music.163.com/user/home?id=482487093'

# 打开链接，获取响应
session = requests.session()
response = session.get(url,headers=headers,cookies=cookies,timeout=(30,60))
# 获取响应的html
html = response.text
# 解析html
soup = BeautifulSoup(html, 'html.parser')
if_user = soup.div['data-module']
if (if_user == 'user'):
    print('He/She is an user!')
else:
    print('Not User, 404 !')

He/She is an user!


### 生成高位随机数

#### 10位号码的前序3位分层随机数生成

假设网易云音乐账户的ID都是10位数，但高位号码未必被完全占用，因此我们需要对高位号码进行测试，看究竟被占用的高位号码有哪些。
因此，我们的思路如下：对10位数中的前3位数（000-999）的每一个数，为其添加100个7位随机数后缀，然后测试这100,000个号码有哪些是真实用户。
这里我们为了测试方便，仅对000-029这30个数进行测试。

In [27]:
prefixes = list(range(30))
randoms_000_029 = []
for prefix in prefixes:
    # 生成7位随机数后缀
    randoms_with_prefix = [int(str(prefix) + str(random.randint(0, 9999999)).zfill(7)) for _ in range(100)]
    # 组合最高位和后缀
    randoms_000_029.extend(randoms_with_prefix)
print(randoms_000_029)

[6873592, 6318407, 7864170, 5034011, 7540996, 5634843, 6857035, 2565836, 981879, 9579587, 3882953, 1874231, 1652527, 9232335, 4167620, 7937744, 1966384, 4029620, 8524415, 6902268, 4292544, 4172825, 1076302, 4521084, 3631474, 4176693, 3239110, 7891544, 3603860, 5674622, 8333406, 1802084, 9164293, 7344356, 3976603, 1512766, 8954494, 282961, 9643552, 4642176, 3601639, 1054209, 9377528, 7101160, 4154186, 341108, 8563386, 4496012, 3473479, 2453392, 7421146, 3089593, 7122586, 3797165, 9120905, 1851419, 7865344, 9921639, 6788308, 5520074, 8935445, 4776739, 1048220, 7891271, 6859660, 3267136, 5777753, 9909558, 2189606, 5116799, 4712750, 6146930, 5029782, 4882821, 5340309, 8189675, 7542963, 5739098, 2465135, 3628818, 259239, 6844942, 1218727, 310244, 487264, 8984850, 9018851, 8159149, 8660005, 6814129, 4640322, 6868194, 2045495, 7685331, 1079805, 4830782, 6068871, 1386515, 7269432, 7359877, 18451888, 16697105, 15412961, 16111994, 17769267, 13131374, 15857426, 18915722, 16402168, 16680057, 10309

### 匹配ID是否为真实用户（高位随机数）

In [28]:
# url-preflix
url_pre = 'https://music.163.com/user/home?id='

# 你随机生成后的ID list
users_id = randoms_000_029
true_users = []
fake_users = []

# 循环检查每个随机生成的“用户”页面
for i in range(len(users_id)):
    
    # 拼接成一个完整的url
    this_url = url_pre + str(users_id[i])
    
    # 打开链接，获取响应
    session = requests.session()
    response = session.get(this_url,headers=headers,cookies=cookies,timeout=(30,60))
    # 获取响应的html
    html = response.text
    # 解析html
    soup = BeautifulSoup(html, 'html.parser')
    
    if_user = soup.div['data-module']
    # 判断是否是真实的user
    if (if_user == 'user'):
        print('He/She is an user!')
        true_users.append(users_id[i])
    else:
        print('Not User, 404!')
        fake_users.append(users_id[i])

print('True users are : {}'.format(true_users))
print('Fake users are : {}'.format(fake_users))

Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
He/She is an user!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not

### 生成中位随机数

接下来，我们正式开始做标志重捕法抽样。因为高位号码被占据的部分往往较少，
因此我们对10位ID的前6位进行分层抽样，后4位做随机抽样。理论上来说，在第一步的基础上，我们已经确定了最高位可能占据的部分，因此这一步随机数生成过程的数值上限是已经被确定的。
重申一下我们的思路：对10位数中的前6位数（000000-高位上限）的每一个数，为其添加5个4位随机数后缀，然后测试这些号码有哪些是真实用户。
这里我们为了测试方便，仅对000300-000599这300个数进行测试。

In [30]:
prefixes = list(range(300, 600))
randoms_000300_000599 = []
for prefix in prefixes:
    # 生成4位随机数后缀
    randoms_with_prefix = [int(str(prefix) + str(random.randint(0, 9999)).zfill(4)) for _ in range(5)]
    # 组合最高位和后缀
    randoms_000300_000599.extend(randoms_with_prefix)
print(randoms_000300_000599)

[3002104, 3004938, 3009867, 3003069, 3005007, 3013449, 3010196, 3018128, 3016762, 3015753, 3022779, 3029189, 3022700, 3028892, 3025181, 3034172, 3033014, 3034453, 3039007, 3038266, 3049211, 3048083, 3044918, 3040801, 3042307, 3054793, 3050493, 3055688, 3051170, 3054379, 3061625, 3066916, 3066802, 3061799, 3060752, 3077357, 3078535, 3076688, 3075783, 3079887, 3083827, 3081866, 3087223, 3080309, 3087030, 3090567, 3096965, 3093744, 3096016, 3094368, 3102844, 3109760, 3108749, 3105815, 3104177, 3117912, 3114609, 3111929, 3119305, 3116528, 3125792, 3128125, 3129320, 3128058, 3123966, 3131105, 3135624, 3133112, 3134456, 3138069, 3143120, 3147697, 3143414, 3145720, 3140569, 3150328, 3152754, 3157132, 3152291, 3157526, 3163963, 3164839, 3163200, 3165142, 3161563, 3179358, 3174214, 3170379, 3177381, 3176398, 3183019, 3189415, 3188053, 3187424, 3181354, 3195904, 3191956, 3197969, 3197269, 3192339, 3204299, 3208925, 3206226, 3206826, 3209124, 3216615, 3212240, 3212991, 3219612, 3213382, 3229559, 

### 匹配ID是否为真实用户（中位随机数）

In [32]:
# url-preflix
url_pre = 'https://music.163.com/user/home?id='

# 你随机生成后的ID list
users_id = randoms_000300_000599
true_users_2 = []
fake_users_2 = []

# 循环检查每个随机生成的“用户”页面
for i in range(len(users_id)):
    
    # 拼接成一个完整的url
    this_url = url_pre + str(users_id[i])
    
    # 打开链接，获取响应
    session = requests.session()
    response = session.get(this_url,headers=headers,cookies=cookies,timeout=(30,60))
    # 获取响应的html
    html = response.text
    # 解析html
    soup = BeautifulSoup(html, 'html.parser')
    
    if_user = soup.div['data-module']
    # 判断是否是真实的user
    if (if_user == 'user'):
        print('He/She is an user!')
        true_users_2.append(users_id[i])
    else:
        print('Not User, 404!')
        fake_users_2.append(users_id[i])

print('True users are : {}'.format(true_users_2))
print('Fake users are : {}'.format(fake_users_2))

Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
He/She is an user!
Not User, 404!
He/She is an user!
He/She is an user!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
He/She is an user!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
He/She is an user!
He/She is an user!
He/She is an user!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
He/She is an user!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
Not User, 404!
No